In [1]:
import polars as pl

In [8]:
csv_files = [
    "allplayers",
    "batting",
    "discreps",
    "ejections",
    "fielding",
    "gameinfo",
    "pitching",
    "teamstats",
]

for csv_file in csv_files:
    df = pl.scan_csv(f"csvs/{csv_file}.csv", infer_schema_length=None)
    df.collect().write_parquet(f"parquets/{csv_file}.parquet")

In [9]:
# Speed test
import time

time_start = time.perf_counter()
pl.scan_csv("csvs/batting.csv", infer_schema_length=None).collect()
print(f"CSV: {time.perf_counter() - time_start:.2f}s")

time_start = time.perf_counter()
pl.scan_parquet("parquets/batting.parquet").collect()
print(f"Parquet: {time.perf_counter() - time_start:.2f}s")


CSV: 10.86s
Parquet: 0.20s
